In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
from google.colab import files

uploaded = files.upload()

json_filename = list(uploaded.keys())[0]

with open(json_filename, 'r', encoding='utf-8') as f:
    data = json.load(f)

print(data)

Saving 논문요약20231006_Validation.json to 논문요약20231006_Validation.json
Buffered data was truncated after reaching the output size limit.

In [ ]:
import shutil
# Google Drive에 저장할 경로 설정
drive_path = '/content/drive/MyDrive/' + json_filename

# Google Drive에 파일 저장
shutil.copy(json_filename, drive_path)

print(f"JSON 파일이 Google Drive에 저장되었습니다: {drive_path}")

JSON 파일이 Google Drive에 저장되었습니다: /content/drive/MyDrive/논문요약20231006_3.json


In [ ]:
import json
import pandas as pd

data_path_list = ["/content/drive/MyDrive/논문요약20231006_0.json", "/content/drive/MyDrive/논문요약20231006_1.json",
                  "/content/drive/MyDrive/논문요약20231006_2.json","/content/drive/MyDrive/논문요약20231006_3.json"]

extracted_data = []

for json_file in data_path_list:
  with open(json_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

    for entry in data[0]['data']:
      doc_id = entry.get("doc_id")
      title = entry.get("title")

      # summary_entire 처리
      for entire in entry.get("summary_entire", []):
        extracted_data.append({
            "doc_id": doc_id,
            "title": title,
            "type": "entire",
            "original_text": entire.get("orginal_text"),
            "summary_text": entire.get("summary_text")
        })

      # sumamry_section 처리
      for section in entry.get("summary_section", []):
        extracted_data.append({
            "doc_id": doc_id,
            "title": title,
            "type": "section",
            "original_text": entire.get("orginal_text"),
            "summary_text": entire.get("summary_text")
        })

df = pd.DataFrame(extracted_data)

In [ ]:
val_path = "/content/drive/MyDrive/논문요약20231006_Validation.json"

extracted_data = []

try:
    with open(val_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

        for entry in data[0]['data']:
            doc_id = entry.get("doc_id")
            title = entry.get("title")

            # summary_entire 처리
            for entire in entry.get("summary_entire", []):
                extracted_data.append({
                    "doc_id": doc_id,
                    "title": title,
                    "type": "entire",
                    "original_text": entire.get("orginal_text"),
                    "summary_text": entire.get("summary_text")
                })

            # summary_section 처리
            for section in entry.get("summary_section", []):
                extracted_data.append({
                    "doc_id": doc_id,
                    "title": title,
                    "type": "section",
                    "original_text": section.get("orginal_text"),  # 수정된 부분
                    "summary_text": section.get("summary_text")  # 수정된 부분
                })

    # DataFrame으로 변환
    df = pd.DataFrame(extracted_data)
    print(df)  # 데이터 확인
except FileNotFoundError:
    print(f"파일을 찾을 수 없습니다: {val_path}")
except Exception as e:
    print(f"오류 발생: {e}")

              doc_id                                              title  \
0      A200928004902                            몰입(Flow) 이론을 적용한 진로상담모형   
1      A200928004902                            몰입(Flow) 이론을 적용한 진로상담모형   
2      A200928004903  지역사회 청소년 상담기관의 실제와 연계체제 활성화 방안에 관한 현장 연구: 안양지역...   
3      A200928004903  지역사회 청소년 상담기관의 실제와 연계체제 활성화 방안에 관한 현장 연구: 안양지역...   
4      A200928005043                         아동의 자살 생각에 영향을 미치는 가족 위험요인   
...              ...                                                ...   
35995  A200928004760                       한국-필리핀 간 FTA 추진에 따른 발전 방안 연구   
35996  A200928004762           해외자본유입이 경제성장에 미치는 영향 분석 - 라틴아메리카를 중심으로 -   
35997  A200928004762           해외자본유입이 경제성장에 미치는 영향 분석 - 라틴아메리카를 중심으로 -   
35998  A200928004764                한국과 라오스 간 경협 확대에 관한 산업 동향과 대응 방안 연구   
35999  A200928004764                한국과 라오스 간 경협 확대에 관한 산업 동향과 대응 방안 연구   

          type                                      original_text  \
0       entire  몰입이론은 원래 창조적인 

In [ ]:
train_df = df[['original_text','summary_text']]
val_df = df[['original_text','summary_text']]

In [ ]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration, Trainer, TrainingArguments

tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/109 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


pytorch_model.bin:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [ ]:
# 토큰화 함수
def tokenize_function(examples):
    model_inputs = tokenizer(examples['original_text'], padding="max_length", truncation=True, max_length=512)

    # labels에 대해서도 패딩 적용
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['summary_text'], padding="max_length", truncation=True, max_length=512)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_train_dataset = dataset.map(tokenize_function, batched=True)

tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

In [ ]:
# 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir = "./results", # 결과 저장 경로
    run_name="my_training_run",
    evaluation_strategy = "epoch", # 평가 전략 설정
    learning_rate = 5e-5, # 학습률 설정
    per_device_train_batch_size = 10, # 배치 크기 설정
    num_train_epochs = 1, # 에폭 수 설정
    weight_decay = 0.01, # 가중치 감소
    report_to="none",
)

# Trainer 설정
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_train_dataset,
    eval_dataset = tokenized_val_dataset,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# 모델 학습
# wandb 비활성화
import os
os.environ["WANDB_DISABLED"] = "true"

trainer.train()

Epoch,Training Loss,Validation Loss
1,0.105100,0.086556


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-defau

TrainOutput(global_step=3600, training_loss=0.14769584814707437, metrics={'train_runtime': 6455.1233, 'train_samples_per_second': 5.577, 'train_steps_per_second': 0.558, 'total_flos': 1.097525624832e+16, 'train_loss': 0.14769584814707437, 'epoch': 1.0})

In [ ]:
save_path = '/content/drive/My Drive/models/kobart_finetuned'

In [ ]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

# 모델과 토크나이저 저장
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}


('/content/drive/My Drive/models/kobart_finetuned/tokenizer_config.json',
 '/content/drive/My Drive/models/kobart_finetuned/special_tokens_map.json',
 '/content/drive/My Drive/models/kobart_finetuned/tokenizer.json')

In [ ]:
# 논문 요약 테스트
from transformers import BartForConditionalGeneration, PreTrainedTokenizerBase

load_path = "/content/drive/MyDrive/models/kobart_finetuned"

model = BartForConditionalGeneration.from_pretrained(load_path)
tokenizer = PreTrainedTokenizerFast.from_pretrained(load_path)

input_text = """"eruginosa 콘택트렌즈 착용 목적은 시력 교정과 미용상의 이유로 젊은층에서 안경을 대신하여 사용빈도가 꾸준히 증가하고 있다(4,7,
11). 콘택트렌즈는 경성 콘택트렌즈와 연성 콘택트렌즈로나뉘며 연성 콘택트렌즈의 종류는 연속 착용렌즈(extended softcontact
lens), 겸용 착용렌즈(flexible soft contact lens), 일일착용렌즈(daily soft contact lens), 그리고
일회용 렌즈(dispo-sable soft contact lens)로 구분이 되고 일회용 렌즈(disposablesoft contact
lens)는 사용기간에 따라 한달 착용렌즈(one-month disposable contact lens), 2주 착용렌즈(two-weeks
disposablecontact lens), 그리고 일일 착용렌즈(one-day disposable contactlens)로 구분된다(12).
생활패턴의 변화로 인해 쉽게 사용하고버릴 수 있는 일회용 렌즈의 사용인원이 다른 콘택트렌즈 사용자에 비해 현저히 증가하고 있는 실정이다. 그러나
일회용 렌즈는 사용과 관리의 편리함으로 인해 젊은 세대에 많은 수요가있으나 장기 사용으로 인한 안 질환이 빈번히 발생하고 있으며그 원인으로는
기존 렌즈에 비해 산소 투과율이 낮아 저산소증을 유발함으로 다양한 감염성 안 질환을 일으키는 것으로 알려져 있다(14, 16). 병원을 내원하는
많은 안질 환자를 대상으로 병원성 세균을 분리동정한 결과 콘택트렌즈 착용자에서 많은 질병이 유발되었으며 그 중 Pseudomonas
aeruginosa 확진에 의한 각막궤양환자20례가 조사되었다(12). 연성 콘택트렌즈 착용으로 인한 각막염의 주요 원인균은
Staphylococcus aureus, P. aeruginosa와Enterobacter aerogenes 등이며 이런 감염성 각막염은
심각할경우 시력상실로 이어지고 있다(7). 외안부 감염의 원인균으로Moraxella lacunata가 보고되었으며(8) 과거 콘택트 렌즈 착용자
중에서 아메바성 각막염이 많은 부분을 차지하였으며 이런이유로 인해 콘택트 렌즈 착용과 아메바성 각막염과의 관련성에 대해 여러 연구진에 의해
조사되었다(4, 13). Acanthamoeba성 질환은 생명을 위협하는 육아종성 아메바 뇌염과 실명을 유발하는 아메바성 각막염을 일으키는
것으로 알려져 있다(4).콘택트 렌즈 사용자 중 진균 감염증의 대부분은 Candidaalbicans에 의한 각막염으로 알려져 있으며 원인으로는
불결한위생이나 부적절한 소독으로 렌즈나 렌즈케이스 내에 진균이증식하는 것으로 조사되었다(1). 많은 연구진들이 연성 콘택트렌즈 사용자들을
대상으로 보존케이스 교환기간, 소독방법, 세척액 종류와 보존케이스 헹굼 여부에 대해 조사하였으며 조사결과, 다양한 안 질환을 일으키는 원인은
콘택트렌즈 사용자들이 렌즈 취급시 불결한 위생과 부적당한 소독 등으로 조사되었다. 최근 렌즈 종류별 선호도를 조사해 본 결과 가장 많이 이용하는
렌즈는 일일 착용렌즈로 48.8%, 그 다음으로 장기 착용렌즈 40%, 1개월 착용렌즈 6.2%. 2주 착용렌즈 2.5%, 난시전용렌즈
2.5%의 순으로 나타났다(17). 그러나 소비층은 경제적인 이유로 일일 착용렌즈를 연속적으로 사용하고 있으며 일일 착용렌즈의 연속적인
사용기간과 연속사용에 따른 세척액종류에 대해서는 전혀 조사된 바가 없다. 이런 이유로 본 연구진은 일회용 렌즈 중에서 일일 착용렌즈의 착용시간을
달리하고, 렌즈 세척액으로 식염수와 보존액을 구분하여 미생물 오염정도 및 오염균을 조사하고자 한다. 대상 및 방법 대상 2009년 대구시 D
대학의 재학생 중에서 콘택트 렌즈를 착용한 경험이 있는 학생을 대상으로 실시하였다. 먼저 지원대상자의 안 검사를 실시하였으며 그 결과 최근
6개월 내 안질환,가려움, 통증 등의 증상이 없는 지원자를 대상으로 하였다. 실험군으로는 일회용 콘택트렌즈 중에서 일일 착용렌즈를 1, 3,5,
10 그리고 15일을 착용하도록 하였고 렌즈 세척액은 ReNuSolution (Bausch and Lomb Company, USA)을 사용한
그룹과 생리식염수를 사용한 그룹으로 나누었다. 각 그룹별로 10명씩 정하여 실험을 실시하였다. 병원성 세균의 분리 배양 일일 착용렌즈를 1일
착용 후 렌즈케이스에 렌즈를 보관한다. 익일 아침 렌즈 착용 후 케이스를 회수하고 회수한 렌즈케 이스 내 미생물의 분포를 조사하고자 하였다.
이런 방법으로각각 3, 5, 10 그리고 15일 착용 후 익일 아침에는 렌즈케이스를 모두 회수하여 렌즈와 렌즈케이스에서 증식하는 세균을
배양하고자 렌즈케이스 내 식염수와 ReNu액을 E-tube에 담은후 3,000 rpm, 10분간 원심 분리한다. 원심분리 후 여액은 버리고
침사층을 회수하여 Brain Heart Infusion (BHI) agar(Becton, Dickinson and Company)에 도말하였고
그람음성세균 분리배양을 목적으로 MacConkey agar (Becton, Dickinsonand Company, USA),
Salmonella-Shigella (S-S) agar (Becton,Dickinson and Company, USA)에 각각 침사층을
도말하여35°C 배양기에서 18-24시간 배양하였다. 렌즈케이스는 멸균된면봉으로 골고루 닦은 다음 BHI agar에 도말하였고 그람음성세균
분리배양을 목적으로 MacConkey agar, S-S agar에 각각도말하여 35°C 배양기에서 18-24시간 배양하였다. 그람양성세균의
분리를 위해 BHI agar에서 배양된 세균집락을 형태에 따라 임의로 집락을 선정하여 BHI agar에 계대 배양하였다(8,15). 병원성
세균의 동정 순수 분리된 균들에 대해 그람 염색을 실시한 후, 그람양성균으로 확인된 균들은 Catalase, Coagulase test를
실시하고 신속간이 동정 키트인 API Staph kit (API system bioMérieuxVitek, Inc., USA)를 이용하여
동정하였다. 표준균주로는 S. MacConkey agar 및 S-S agar에서 배양된 균들도 각 검체마다 집락을 선정하여 그람 염색 후,
단일집락을 분리 배양하여 API 20 E kit, API 20 NE kit로 동정하였다. API web site에 연결해 동정결과를 확인하였다.
결과 일일 착용렌즈 착용시 렌즈 세척액별, 사용기간별에 따른 세균 분포 실험군으로는 일일 착용렌즈를 1, 3, 5, 10 그리고
15일을착용하도록 하였고 렌즈 세척액은 ReNu를 사용한 그룹과 생리식염수를 사용한 그룹으로 나누었다. 각 그룹별로 10명씩 정하여 실험을
실시하였다. 식염수를 사용하는 그룹과 ReNu를사용하는 그룹에서 세균의 분포를 조사하기 위해 렌즈케이스내 여액과 렌즈케이스를 멸균된 면봉으로
닦은 후 BHI agar,MacConkey agar, S-S agar에 도말 하였다. 식염수 사용자에서세균분포를 조사한 결과, 1일 10개
렌즈케이스 중 8개(80%), 3일 10개 중 8개(80%), 5일 10개 중 9개(90%), 10일 10개 중 9개(90%) 그리고 15일은
10개 중 9개(90%)에서 세균의 증식을관찰하였고 ReNu를 사용한 결과 1일 10개 중 4개(40%), 3일10개 중 4개(40%), 5일
10개 중 5개(50%), 10일 10개 중 4개 154 Choi et al."""

inputs = tokenizer(input_text, max_length=512, truncation=True, return_tensors='pt')

# 요약 생성
summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=150, early_stopping=True)
generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("생성된 요약:", generated_summary)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


생성된 요약: 콘택트렌즈는 경성 콘택트렌즈와 연성 콘택트렌즈로나뉘며 연성 콘택트렌즈의 종류는 연속 착용렌즈, 겸용 착용렌즈, 일일착용렌즈, 일일착용렌즈, 그리고 일회용 렌즈로 구분이 되고 일회용 렌즈는 사용기간에 따라 한달 착용렌즈(one-month disposable contact lens), 2주 착용렌즈(two-weeks), 그리고 일일 착용렌즈(one-day disposable contactlens), 그리고 일일 착용렌즈(one-day dis


In [ ]:
# 텍스트를 일정한 길이로 분할
text_parts = [input_text[i:i+400] for i in range(0, len(input_text), 400)]

summaries = []
for part in text_parts:
    inputs = tokenizer(part, max_length=512, truncation=True, return_tensors='pt')
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=150, early_stopping=True)
    part_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    summaries.append(part_summary)

combined_summary = " ".join(summaries)
inputs = tokenizer(combined_summary, max_length=512, truncation=True, return_tensors='pt')
final_summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=150, early_stopping=True)
final_summary = tokenizer.decode(final_summary_ids[0], skip_special_tokens=True)

print("최종 요약:", final_summary)

최종 요약: 콘택트렌즈는 경성 콘택트렌즈와 연성 콘택트렌즈로나뉘며 연성 콘택트렌즈의 종류는 연속 착용렌즈(extended softcontact 201lens), 겸용 착용렌즈(flexible soft contact lens), 일일착용렌즈(daily soft contact lens), 일회용 렌즈(disposablesoft contac 일회용 렌즈는 사용과 관리의 편리함으로 인해 젊은 세대에 많은 수요가있으나 장기 사용으로 인한 안 질환이 빈번히


In [ ]:
# 텍스트를 일정한 길이로 분할
text_parts = [input_text[i:i+400] for i in range(0, len(input_text), 400)]

summaries = []
for part in text_parts:
    inputs = tokenizer(part, max_length=512, truncation=True, return_tensors='pt')
    summary_ids = model.generate(
        inputs['input_ids'],
        num_beams=10,
        max_length=200,       # 더 긴 요약을 위해 max_length를 200으로 설정
        min_length=100,       # 최소 요약 길이를 설정해 너무 짧은 요약 방지
        length_penalty=2.0,   # 길이를 더 선호하도록 설정
        early_stopping=True
    )
    part_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    summaries.append(part_summary)

# 부분 요약을 결합
combined_summary = " ".join(summaries)
inputs = tokenizer(combined_summary, max_length=512, truncation=True, return_tensors='pt')

# 최종 요약 생성
final_summary_ids = model.generate(
    inputs['input_ids'],
    num_beams=4,
    max_length=200,           # 최종 요약의 길이 설정
    min_length=100,           # 최종 요약의 최소 길이 설정
    length_penalty=2.0,       # 길이를 선호하도록 설정
    early_stopping=True
)
final_summary = tokenizer.decode(final_summary_ids[0], skip_special_tokens=True)

print("최종 요약:", final_summary)


최종 요약: 일회용 렌즈는 사용과 관리의 편리함으로 인해 젊은 세대에 많은 수요가있으나 장기 사용으로 인한 안 질환이 빈번히 발생하고 있으며 그 원인으로는 기존 렌즈에 비해 산소 투과율이 낮아 저산소증을 유발함으로 다양한 감염성 안 질환을 일으키는 것으로 알려져 있다.
연성 콘택트렌즈 착용으로 인한 각막염의 주요 원인균은 Staphylococcus aureus, P. aeruginosa와Enterobacter aerogenes 등이며 감염성 각막염은 심각할경우 시력상실로 이어지고 있다.


In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=7a7124f68f42b7acdc04b547e403f6073fcc7f774b87bb2decd680f49978d9cc
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 13.7 MB/s eta 0:00:00


In [ ]:
from rouge_score import rouge_scorer
from konlpy.tag import Okt

# 원본 최종 요약 (모델이 생성한 요약)
model_summary = """
일회용 렌즈는 사용과 관리의 편리함으로 인해 젊은 세대에 많은 수요가있으나 장기 사용으로 인한 안 질환이 빈번히 발생하고 있으며 그 원인으로는 기존 렌즈에 비해 산소 투과율이 낮아 저산소증을 유발함으로 다양한 감염성 안 질환을 일으키는 것으로 알려져 있다.
연성 콘택트렌즈 착용으로 인한 각막염의 주요 원인균은 Staphylococcus aureus, P. aeruginosa와Enterobacter aerogenes 등이며 감염성 각막염은 심각할경우 시력상실로 이어지고 있다.
"""

# 제공된 요약 (제가 작성한 요약)
provided_summary = """
이 논문은 일회용 콘택트렌즈 장기 착용 시 미생물 오염과 원인균을 조사한 연구로, 편리함 때문에 사용이 증가했으나 낮은 산소 투과율로 안 질환 위험이 높아진다는 결과를 보였다. 특히, 식염수 사용 그룹에서 세균 증식이 많았으며, ReNu 용액 사용 시 오염이 감소하는 경향을 보여 콘택트렌즈 관리 방식이 감염 예방에 중요함을 시사한다.
"""

# 전처리 함수
def preprocess(text):
    okt = Okt()
    tokens = okt.morphs(text, stem=True)  # 형태소 분리 및 어근화
    stop_words = set(['을', '를', '이', '가', '에', '은', '는', '으로', '하다'])  # 불용어 예시
    return " ".join([word for word in tokens if word not in stop_words])

# 원본 요약과 제공된 요약을 전처리
model_summary_processed = preprocess(model_summary)
provided_summary_processed = preprocess(provided_summary)

# ROUGE 평가자 초기화
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# ROUGE 점수 계산
scores = scorer.score(provided_summary_processed, model_summary_processed)

# 결과 출력
print("ROUGE-1:", scores['rouge1'])
print("ROUGE-2:", scores['rouge2'])
print("ROUGE-L:", scores['rougeL'])


ROUGE-1: Score(precision=0.0, recall=0.0, fmeasure=0.0)
ROUGE-2: Score(precision=0.0, recall=0.0, fmeasure=0.0)
ROUGE-L: Score(precision=0.0, recall=0.0, fmeasure=0.0)


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=82726dfc336b21b10ffdcd8e2ee7320a6334206981e154ba8437d603f90108b4
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 24.8 MB/s eta 0:00:00


In [ ]:
from rouge_score import rouge_scorer
from konlpy.tag import Okt

# 원본 최종 요약 (모델이 생성한 요약)
model_summary = """
일회용 렌즈는 사용과 관리의 편리함으로 인해 젊은 세대에 많은 수요가있으나 장기 사용으로 인한 안 질환이 빈번히 발생하고 있으며 그 원인으로는 기존 렌즈에 비해 산소 투과율이 낮아 저산소증을 유발함으로 다양한 감염성 안 질환을 일으키는 것으로 알려져 있다.
연성 콘택트렌즈 착용으로 인한 각막염의 주요 원인균은 Staphylococcus aureus, P. aeruginosa와Enterobacter aerogenes 등이며 감염성 각막염은 심각할경우 시력상실로 이어지고 있다.
"""

# 제공된 요약 (제가 작성한 요약)
provided_summary = """
이 논문은 일회용 콘택트렌즈 장기 착용 시 미생물 오염과 원인균을 조사한 연구로, 편리함 때문에 사용이 증가했으나 낮은 산소 투과율로 안 질환 위험이 높아진다는 결과를 보였다. 특히, 식염수 사용 그룹에서 세균 증식이 많았으며, ReNu 용액 사용 시 오염이 감소하는 경향을 보여 콘택트렌즈 관리 방식이 감염 예방에 중요함을 시사한다.
"""

# 전처리 함수 (토큰 리스트 형태 반환)
def preprocess(text):
    okt = Okt()
    tokens = okt.morphs(text, stem=True)  # 형태소 분리 및 어근화
    stop_words = set(['을', '를', '이', '가', '에', '은', '는', '으로', '하다'])  # 불용어 예시
    return [word for word in tokens if word not in stop_words]

# 원본 요약과 제공된 요약을 전처리
model_summary_tokens = preprocess(model_summary)
provided_summary_tokens = preprocess(provided_summary)

# 토큰을 다시 문자열로 변환 (리스트를 공백으로 결합하여 문자열 형태로 전달)
model_summary_processed = " ".join(model_summary_tokens)
provided_summary_processed = " ".join(provided_summary_tokens)

# ROUGE 평가자 초기화
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# ROUGE 점수 계산
scores = scorer.score(provided_summary_processed, model_summary_processed)

# 결과 출력
print("ROUGE-1:", scores['rouge1'])
print("ROUGE-2:", scores['rouge2'])
print("ROUGE-L:", scores['rougeL'])


ROUGE-1: Score(precision=0.0, recall=0.0, fmeasure=0.0)
ROUGE-2: Score(precision=0.0, recall=0.0, fmeasure=0.0)
ROUGE-L: Score(precision=0.0, recall=0.0, fmeasure=0.0)


In [ ]:
from konlpy.tag import Okt
from collections import Counter

# 원본 최종 요약 (모델이 생성한 요약)
model_summary = """
일회용 렌즈는 사용과 관리의 편리함으로 인해 젊은 세대에 많은 수요가 있으나 장기 사용으로 인한 안 질환이 빈번히 발생하고 있으며 그 원인으로는 기존 렌즈에 비해 산소 투과율이 낮아 저산소증을 유발하여 다양한 감염성 안 질환을 일으키는 것으로 알려져 있다.
연성 콘택트렌즈 착용으로 인한 각막염의 주요 원인균은 Staphylococcus aureus, P. aeruginosa와 Enterobacter aerogenes 등이며 감염성 각막염은 심각할 경우 시력 상실로 이어지고 있다.
"""

# 제공된 요약 (제가 작성한 요약)
provided_summary = """
이 논문은 일회용 콘택트렌즈 장기 착용 시 미생물 오염과 원인균을 조사한 연구로, 편리함 때문에 사용이 증가했으나 낮은 산소 투과율로 안 질환 위험이 높아진다는 결과를 보였다. 특히, 식염수 사용 그룹에서 세균 증식이 많았으며, ReNu 용액 사용 시 오염이 감소하는 경향을 보여 콘택트렌즈 관리 방식이 감염 예방에 중요함을 시사한다.
"""

# 전처리 함수
def preprocess(text):
    okt = Okt()
    tokens = okt.morphs(text, stem=True)  # 형태소 분리 및 어근화
    stop_words = set(['을', '를', '이', '가', '에', '은', '는', '으로', '하다'])  # 불용어 예시
    return [word for word in tokens if word not in stop_words]

# n-gram 추출 함수
def get_ngrams(tokens, n):
    return [tuple(tokens[i:i + n]) for i in range(len(tokens) - n + 1)]

# ROUGE 계산 함수
def calculate_rouge(candidate, reference, n):
    candidate_ngrams = Counter(get_ngrams(candidate, n))
    reference_ngrams = Counter(get_ngrams(reference, n))

    overlap = sum((candidate_ngrams & reference_ngrams).values())
    recall = overlap / sum(reference_ngrams.values())
    precision = overlap / sum(candidate_ngrams.values())
    f1_score = 2 * ((precision * recall) / (precision + recall + 1e-8))

    return {"precision": precision, "recall": recall, "f1": f1_score}

# LCS (Longest Common Subsequence) 계산 함수
def lcs(X, Y):
    m = len(X)
    n = len(Y)
    L = [[0] * (n + 1) for _ in range(m + 1)]
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0:
                L[i][j] = 0
            elif X[i - 1] == Y[j - 1]:
                L[i][j] = L[i - 1][j - 1] + 1
            else:
                L[i][j] = max(L[i - 1][j], L[i][j - 1])
    return L[m][n]

# ROUGE-L 계산 함수
def calculate_rouge_l(candidate, reference):
    lcs_length = lcs(candidate, reference)
    recall = lcs_length / len(reference)
    precision = lcs_length / len(candidate)
    f1_score = 2 * ((precision * recall) / (precision + recall + 1e-8))
    return {"precision": precision, "recall": recall, "f1": f1_score}

# 전처리
model_summary_tokens = preprocess(model_summary)
provided_summary_tokens = preprocess(provided_summary)

# ROUGE-1, ROUGE-2, ROUGE-L 계산
rouge1 = calculate_rouge(provided_summary_tokens, model_summary_tokens, 1)
rouge2 = calculate_rouge(provided_summary_tokens, model_summary_tokens, 2)
rougeL = calculate_rouge_l(provided_summary_tokens, model_summary_tokens)

# 결과 출력
print("ROUGE-1:", rouge1)
print("ROUGE-2:", rouge2)
print("ROUGE-L:", rougeL)


ROUGE-1: {'precision': 0.4032258064516129, 'recall': 0.3125, 'f1': 0.35211267113667927}
ROUGE-2: {'precision': 0.08196721311475409, 'recall': 0.06329113924050633, 'f1': 0.07142856651122484}
ROUGE-L: {'precision': 0.24193548387096775, 'recall': 0.1875, 'f1': 0.21126760071414413}


In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# 문장 임베딩 계산
embeddings1 = model.encode(model_summary, convert_to_tensor=True)
embeddings2 = model.encode(provided_summary, convert_to_tensor=True)

# 코사인 유사도 계산
cosine_similarity = util.cos_sim(embeddings1, embeddings2)

print("코사인 유사도:", cosine_similarity.item())

코사인 유사도: 0.7147229909896851


In [ ]:
from itertools import combinations
from konlpy.tag import Okt

# 전처리 함수
def preprocess(text):
    okt = Okt()
    tokens = okt.morphs(text)  # 형태소 분석을 통해 토큰화
    return tokens  # 중복 제거되지 않은 원본 토큰 반환

# 스킵-2-그램 추출 함수
def get_skip_bigrams(tokens):
    skip_bigrams = set()
    for i, token in enumerate(tokens):
        for j in range(i + 1, len(tokens)):
            if j - i > 2:  # 스킵-2-그램으로 설정 (최대 1개 단어를 건너뜀)
                break
            skip_bigrams.add((tokens[i], tokens[j]))
    return skip_bigrams

# 모델이 생성한 요약문과 제가 작성한 요약문
original_text = """
일회용 렌즈는 사용과 관리의 편리함으로 인해 젊은 세대에 많은 수요가있으나 장기 사용으로 인한 안 질환이 빈번히 발생하고 있으며 그 원인으로는 기존 렌즈에 비해 산소 투과율이 낮아 저산소증을 유발함으로 다양한 감염성 안 질환을 일으키는 것으로 알려져 있다.
연성 콘택트렌즈 착용으로 인한 각막염의 주요 원인균은 Staphylococcus aureus, P. aeruginosa와 Enterobacter aerogenes 등이며 감염성 각막염은 심각할 경우 시력상실로 이어지고 있다.
"""

summary_text = """
이 논문은 일회용 콘택트렌즈 장기 착용 시 미생물 오염과 원인균을 조사한 연구로, 편리함 때문에 사용이 증가했으나 낮은 산소 투과율로 안 질환 위험이 높아진다는 결과를 보였다. 특히, 식염수 사용 그룹에서 세균 증식이 많았으며, ReNu 용액 사용 시 오염이 감소하는 경향을 보여 콘택트렌즈 관리 방식이 감염 예방에 중요함을 시사한다.
"""

# 유니그램 및 스킵-2-그램 추출
original_tokens = preprocess(original_text)
summary_tokens = preprocess(summary_text)

original_unigrams = set(original_tokens)
summary_unigrams = set(summary_tokens)
original_skip_bigrams = get_skip_bigrams(original_tokens)
summary_skip_bigrams = get_skip_bigrams(summary_tokens)

# 유니그램 및 스킵-2-그램 겹침 계산
overlap_unigrams = original_unigrams.intersection(summary_unigrams)
overlap_skip_bigrams = original_skip_bigrams.intersection(summary_skip_bigrams)

# ROUGE-SU 점수: 유니그램과 스킵-2-그램의 평균 비율
rouge_su_score = (len(overlap_unigrams) + len(overlap_skip_bigrams)) / (
    len(original_unigrams) + len(original_skip_bigrams)
)

print("ROUGE-SU Score:", rouge_su_score)


ROUGE-SU Score: 0.1225296442687747


In [ ]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

# 파인튜닝하지 않은 기본 모델 로드
model_name = "gogamza/kobart-base-v2"  # KoBART 기본 모델
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# 요약할 텍스트 입력
input_text = """"eruginosa 콘택트렌즈 착용 목적은 시력 교정과 미용상의 이유로 젊은층에서 안경을 대신하여 사용빈도가 꾸준히 증가하고 있다(4,7,
11). 콘택트렌즈는 경성 콘택트렌즈와 연성 콘택트렌즈로나뉘며 연성 콘택트렌즈의 종류는 연속 착용렌즈(extended softcontact
lens), 겸용 착용렌즈(flexible soft contact lens), 일일착용렌즈(daily soft contact lens), 그리고
일회용 렌즈(dispo-sable soft contact lens)로 구분이 되고 일회용 렌즈(disposablesoft contact
lens)는 사용기간에 따라 한달 착용렌즈(one-month disposable contact lens), 2주 착용렌즈(two-weeks
disposablecontact lens), 그리고 일일 착용렌즈(one-day disposable contactlens)로 구분된다(12).
생활패턴의 변화로 인해 쉽게 사용하고버릴 수 있는 일회용 렌즈의 사용인원이 다른 콘택트렌즈 사용자에 비해 현저히 증가하고 있는 실정이다. 그러나
일회용 렌즈는 사용과 관리의 편리함으로 인해 젊은 세대에 많은 수요가있으나 장기 사용으로 인한 안 질환이 빈번히 발생하고 있으며그 원인으로는
기존 렌즈에 비해 산소 투과율이 낮아 저산소증을 유발함으로 다양한 감염성 안 질환을 일으키는 것으로 알려져 있다(14, 16). 병원을 내원하는
많은 안질 환자를 대상으로 병원성 세균을 분리동정한 결과 콘택트렌즈 착용자에서 많은 질병이 유발되었으며 그 중 Pseudomonas
aeruginosa 확진에 의한 각막궤양환자20례가 조사되었다(12). 연성 콘택트렌즈 착용으로 인한 각막염의 주요 원인균은
Staphylococcus aureus, P. aeruginosa와Enterobacter aerogenes 등이며 이런 감염성 각막염은
심각할경우 시력상실로 이어지고 있다(7). 외안부 감염의 원인균으로Moraxella lacunata가 보고되었으며(8) 과거 콘택트 렌즈 착용자
중에서 아메바성 각막염이 많은 부분을 차지하였으며 이런이유로 인해 콘택트 렌즈 착용과 아메바성 각막염과의 관련성에 대해 여러 연구진에 의해
조사되었다(4, 13). Acanthamoeba성 질환은 생명을 위협하는 육아종성 아메바 뇌염과 실명을 유발하는 아메바성 각막염을 일으키는
것으로 알려져 있다(4).콘택트 렌즈 사용자 중 진균 감염증의 대부분은 Candidaalbicans에 의한 각막염으로 알려져 있으며 원인으로는
불결한위생이나 부적절한 소독으로 렌즈나 렌즈케이스 내에 진균이증식하는 것으로 조사되었다(1). 많은 연구진들이 연성 콘택트렌즈 사용자들을
대상으로 보존케이스 교환기간, 소독방법, 세척액 종류와 보존케이스 헹굼 여부에 대해 조사하였으며 조사결과, 다양한 안 질환을 일으키는 원인은
콘택트렌즈 사용자들이 렌즈 취급시 불결한 위생과 부적당한 소독 등으로 조사되었다. 최근 렌즈 종류별 선호도를 조사해 본 결과 가장 많이 이용하는
렌즈는 일일 착용렌즈로 48.8%, 그 다음으로 장기 착용렌즈 40%, 1개월 착용렌즈 6.2%. 2주 착용렌즈 2.5%, 난시전용렌즈
2.5%의 순으로 나타났다(17). 그러나 소비층은 경제적인 이유로 일일 착용렌즈를 연속적으로 사용하고 있으며 일일 착용렌즈의 연속적인
사용기간과 연속사용에 따른 세척액종류에 대해서는 전혀 조사된 바가 없다. 이런 이유로 본 연구진은 일회용 렌즈 중에서 일일 착용렌즈의 착용시간을
달리하고, 렌즈 세척액으로 식염수와 보존액을 구분하여 미생물 오염정도 및 오염균을 조사하고자 한다. 대상 및 방법 대상 2009년 대구시 D
대학의 재학생 중에서 콘택트 렌즈를 착용한 경험이 있는 학생을 대상으로 실시하였다. 먼저 지원대상자의 안 검사를 실시하였으며 그 결과 최근
6개월 내 안질환,가려움, 통증 등의 증상이 없는 지원자를 대상으로 하였다. 실험군으로는 일회용 콘택트렌즈 중에서 일일 착용렌즈를 1, 3,5,
10 그리고 15일을 착용하도록 하였고 렌즈 세척액은 ReNuSolution (Bausch and Lomb Company, USA)을 사용한
그룹과 생리식염수를 사용한 그룹으로 나누었다. 각 그룹별로 10명씩 정하여 실험을 실시하였다. 병원성 세균의 분리 배양 일일 착용렌즈를 1일
착용 후 렌즈케이스에 렌즈를 보관한다. 익일 아침 렌즈 착용 후 케이스를 회수하고 회수한 렌즈케 이스 내 미생물의 분포를 조사하고자 하였다.
이런 방법으로각각 3, 5, 10 그리고 15일 착용 후 익일 아침에는 렌즈케이스를 모두 회수하여 렌즈와 렌즈케이스에서 증식하는 세균을
배양하고자 렌즈케이스 내 식염수와 ReNu액을 E-tube에 담은후 3,000 rpm, 10분간 원심 분리한다. 원심분리 후 여액은 버리고
침사층을 회수하여 Brain Heart Infusion (BHI) agar(Becton, Dickinson and Company)에 도말하였고
그람음성세균 분리배양을 목적으로 MacConkey agar (Becton, Dickinsonand Company, USA),
Salmonella-Shigella (S-S) agar (Becton,Dickinson and Company, USA)에 각각 침사층을
도말하여35°C 배양기에서 18-24시간 배양하였다. 렌즈케이스는 멸균된면봉으로 골고루 닦은 다음 BHI agar에 도말하였고 그람음성세균
분리배양을 목적으로 MacConkey agar, S-S agar에 각각도말하여 35°C 배양기에서 18-24시간 배양하였다. 그람양성세균의
분리를 위해 BHI agar에서 배양된 세균집락을 형태에 따라 임의로 집락을 선정하여 BHI agar에 계대 배양하였다(8,15). 병원성
세균의 동정 순수 분리된 균들에 대해 그람 염색을 실시한 후, 그람양성균으로 확인된 균들은 Catalase, Coagulase test를
실시하고 신속간이 동정 키트인 API Staph kit (API system bioMérieuxVitek, Inc., USA)를 이용하여
동정하였다. 표준균주로는 S. MacConkey agar 및 S-S agar에서 배양된 균들도 각 검체마다 집락을 선정하여 그람 염색 후,
단일집락을 분리 배양하여 API 20 E kit, API 20 NE kit로 동정하였다. API web site에 연결해 동정결과를 확인하였다.
결과 일일 착용렌즈 착용시 렌즈 세척액별, 사용기간별에 따른 세균 분포 실험군으로는 일일 착용렌즈를 1, 3, 5, 10 그리고
15일을착용하도록 하였고 렌즈 세척액은 ReNu를 사용한 그룹과 생리식염수를 사용한 그룹으로 나누었다. 각 그룹별로 10명씩 정하여 실험을
실시하였다. 식염수를 사용하는 그룹과 ReNu를사용하는 그룹에서 세균의 분포를 조사하기 위해 렌즈케이스내 여액과 렌즈케이스를 멸균된 면봉으로
닦은 후 BHI agar,MacConkey agar, S-S agar에 도말 하였다. 식염수 사용자에서세균분포를 조사한 결과, 1일 10개
렌즈케이스 중 8개(80%), 3일 10개 중 8개(80%), 5일 10개 중 9개(90%), 10일 10개 중 9개(90%) 그리고 15일은
10개 중 9개(90%)에서 세균의 증식을관찰하였고 ReNu를 사용한 결과 1일 10개 중 4개(40%), 3일10개 중 4개(40%), 5일
10개 중 5개(50%), 10일 10개 중 4개 154 Choi et al."""

# 토크나이저로 텍스트 인코딩
inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

# 요약 생성
summary_ids = model.generate(inputs, max_length=150, min_length=50, num_beams=8, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 요약 결과 출력
print("기본 모델 요약 결과:", summary)


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


기본 모델 요약 결과: 이이서 안 안경을 대신하여 사용 빈도가 꾸준히 증가하고 있다(4,7,
11). 콘택트렌즈는 경성 콘택트렌즈와 연성 콘택트렌즈로나뉘며 연성 콘택트렌즈의 종류는 연속 착용렌즈(연성 콘택트렌즈로나뉘며 연성 콘택트렌즈의 종류는 연성 콘택트렌즈의 종류는
